<div class="practice--panel--ejs8J"><div data-purpose="practice-component-text" class="rt-scaffolding"><h4>Demonstrate Direct Data Sharing using SQL</h4><p><br></p><p>This hands-on activity shows how to use SQL to share a table with another Snowflake account. We create a new Share object, add a new table to it, grant the relevant permissions, and make the Share object available to consumers. Then, as the consumer account, we use the share to create a read-only database, from which we may consume the shared data.</p><p><strong>You will need two Snowflake accounts for this exercise, one as a data supplier and one as a data consumer.</strong> We recommend that the two accounts used for this experiment be on the <strong><em>same cloud provider and in the same region</em></strong>. Although Snowflake facilitates data sharing between geographies and cloud providers, it does require data replication, which is beyond the scope of this course.</p><p>Please note that we will use the ACCOUNTADMIN role to create and manage the share. However, it is possible to grant CREATE SHARE privileges to another role, which can create a new share instead of ACCOUNTADMIN.</p><p><br></p><p><strong>The following steps must be run while logged into the <em>provider</em> account.</strong></p><p>1. Let us start by creating a database and a table that we use to demonstrate sharing.</p><div class="ud-component--base-components--code-block"><div><pre class="prettyprint linenums prettyprinted" role="presentation" style=""><ol class="linenums"><li class="L0"><span class="pln">CREATE OR REPLACE DATABASE sharing_demo_db</span><span class="pun">;</span></li><li class="L1"><span class="pln">USE DATABASE sharing_demo_db</span><span class="pun">;</span></li></ol></pre></div></div><p>2. Next, we create a new table and populate it with some data. Let us simplify the process by using the STORE table from the Snowflake sample database. We have named the new table STORE. To create a table from the sample database, use the following SQL.</p><div class="ud-component--base-components--code-block"><div><pre class="prettyprint linenums prettyprinted" role="presentation" style=""><ol class="linenums"><li class="L0"><span class="pln">CREATE TABLE STORE</span></li><li class="L1"><span class="pln">AS SELECT </span><span class="pun">*</span><span class="pln"> FROM SNOWFLAKE_SAMPLE_DATA</span><span class="pun">.</span><span class="pln">TPCDS_SF10TCL</span><span class="pun">.</span><span class="pln">STORE</span><span class="pun">;</span></li></ol></pre></div></div><p><br></p><p>3. Next, switch to the ACCOUNTADMIN role, which has the required privileges to create a Share object.</p><div class="ud-component--base-components--code-block"><div><pre class="prettyprint linenums prettyprinted" role="presentation" style=""><ol class="linenums"><li class="L0"><span class="pln">USE ROLE ACCOUNTADMIN</span><span class="pun">;</span></li></ol></pre></div></div><p>4. Creating a new Share is a straightforward process and can be achieved through the following syntax.</p><div class="ud-component--base-components--code-block"><div><pre class="prettyprint linenums prettyprinted" role="presentation" style=""><ol class="linenums"><li class="L0"><span class="pln">CREATE SHARE shr_STORE</span><span class="pun">;</span></li></ol></pre></div></div><p><br></p><p>5. Next, we add USAGE privilege to the database and the schema containing the table to be shared. The USAGE privileges must be provided to the share before adding the table to the share. To add the USAGE privilege, run the following SQL.</p><div class="ud-component--base-components--code-block"><div><pre class="prettyprint linenums prettyprinted" role="presentation" style=""><ol class="linenums"><li class="L0"><span class="pln">GRANT USAGE ON DATABASE sharing_demo_db TO SHARE shr_STORE</span><span class="pun">;</span></li><li class="L1"><span class="pln">GRANT USAGE ON SCHEMA sharing_demo_db</span><span class="pun">.</span><span class="kwd">public</span><span class="pln"> TO SHARE shr_STORE</span><span class="pun">;</span></li></ol></pre></div></div><p><br></p><p>6. Now we will add the STORE table to this share. To add a table to a share, grant the share SELECT access to the table. The following SQL adds the STORE table to the shr_STORE.</p><div class="ud-component--base-components--code-block"><div><pre class="prettyprint linenums prettyprinted" role="presentation" style=""><ol class="linenums"><li class="L0"><span class="pln">GRANT SELECT ON TABLE sharing_demo_db</span><span class="pun">.</span><span class="kwd">public</span><span class="pun">.</span><span class="pln">STORE TO SHARE shr_STORE</span><span class="pun">;</span></li></ol></pre></div></div><p><br></p><p>7. Finally, as the final step on the producer side, we add one or more consumer accounts to the share. This step requires the account name of the consumer. You can find the account name for a Snowflake account in the new Snowflake web user interface. The bottom left of the screen shows account information, or you can also determine the account name from the URL.</p><div class="ud-component--base-components--code-block"><div><pre class="prettyprint linenums prettyprinted" role="presentation" style=""><ol class="linenums"><li class="L0"><span class="pln">ALTER SHARE shr_STORE ADD ACCOUNT </span><span class="pun">=</span><span class="pln"> </span><span class="str">&lt;consumer_account_name&gt;</span><span class="pun">;</span></li></ol></pre></div></div><p><br></p><p><br></p><p><strong>The following steps must be run while logged into the <em>consumer</em> account.</strong></p><p>Please note that we will use the ACCOUNTADMIN role to perform the following steps. It is possible to grant IMPORT SHARE privileges to another role, which can then consume a share instead of ACCOUNTADMIN.</p><p><br></p><p>8. To consume shared data, you must create a database on the share. The step requires knowledge of the provider account name. Please follow step 7 to determine the producer account name.</p><div class="ud-component--base-components--code-block"><div><pre class="prettyprint linenums prettyprinted" role="presentation" style=""><ol class="linenums"><li class="L0"><span class="pln">USE ROLE ACCOUNTADMIN</span><span class="pun">;</span></li><li class="L1"><span class="pln">CREATE DATABASE </span><span class="typ">Procurement_DB</span><span class="pln"> FROM SHARE </span><span class="str">&lt;provider_account_name&gt;</span><span class="pun">.</span><span class="pln">shr_STORE</span><span class="pun">;</span></li></ol></pre></div></div><p><br></p><p>9. After the database has been created, the shared table appears under the database. As a result, you can query the shared table just like any other table in your system.</p><div class="ud-component--base-components--code-block"><div><pre class="prettyprint linenums prettyprinted" role="presentation" style=""><ol class="linenums"><li class="L0"><span class="pln">SELECT </span><span class="pun">*</span><span class="pln"> FROM </span><span class="typ">Procurement_DB</span><span class="pun">.</span><span class="kwd">public</span><span class="pun">.</span><span class="pln">STORE</span><span class="pun">;</span></li></ol></pre></div></div><p><br></p><p>10. Let us count the number of rows in the table because, in the following steps, we will delete some rows on the provider and validate if the change is reflected in the consumer.</p><div class="ud-component--base-components--code-block"><div><pre class="prettyprint linenums prettyprinted" role="presentation" style=""><ol class="linenums"><li class="L0"><span class="pln">SELECT COUNT</span><span class="pun">(*)</span><span class="pln"> FROM </span><span class="typ">Procurement_DB</span><span class="pun">.</span><span class="kwd">public</span><span class="pun">.</span><span class="pln">STORE</span><span class="pun">;</span></li></ol></pre></div></div><p><br></p><p><strong>The following step must be run while logged into the <em>provider</em> account.</strong></p><p>11. We delete some of the rows from the STORE table. The following DELETE command will affect 77 rows.</p><div class="ud-component--base-components--code-block"><div><pre class="prettyprint linenums prettyprinted" role="presentation" style=""><ol class="linenums"><li class="L0"><span class="pln">USE DATABASE sharing_demo_db</span><span class="pun">;</span></li><li class="L1"><span class="pln">DELETE FROM STORE WHERE S_STATE </span><span class="pun">=</span><span class="pln"> </span><span class="str">'TN'</span><span class="pun">;</span></li></ol></pre></div></div><p><br></p><p>12. Let us find out the new row count in the table. The results of the following query should result in 1,423 being returned.</p><div class="ud-component--base-components--code-block"><div><pre class="prettyprint linenums prettyprinted" role="presentation" style=""><ol class="linenums"><li class="L0"><span class="pln">SELECT COUNT</span><span class="pun">(*)</span><span class="pln"> FROM STORE</span><span class="pun">;</span></li></ol></pre></div></div><p><br></p><p><strong>The following step must be run while logged into the <em>consumer</em> account.</strong></p><p>13. While logged into the consumer account, run the following SQL, which should result in 1,423 being returned. The result indicates that any changes in the table on the provider are automatically reflected on the consumer since the rows deleted on the provider are no longer present in the consumer either.</p><div class="ud-component--base-components--code-block"><div><pre class="prettyprint linenums prettyprinted" role="presentation" style=""><ol class="linenums"><li class="L0"><span class="pln">SELECT COUNT</span><span class="pun">(*)</span><span class="pln"> FROM </span><span class="typ">Procurement_DB</span><span class="pun">.</span><span class="kwd">public</span><span class="pun">.</span><span class="pln">STORE</span><span class="pun">;</span></li></ol></pre></div></div></div><div class="practice--question-list--1HR9g practice--mt-md--3j-aS" data-purpose="question-list"><h4 class="ud-heading-md practice--mb-xs--Lpg-b">Questions for this assignment</h4><div><div class="question--question--152Yi"><div class="ud-text-bold question--title--2FD-U"><div data-purpose="safely-set-inner-html:rich-text-viewer:html" class="question--title-text--2kBEO rt-scaffolding"><p>Will the consumer see the new rows if the provider adds new rows to a shared table?</p></div></div><div class="question--answer--2ASXa"></div></div></div></div></div>

<p>
<p><b>Q1 Ans</b></p>
<p>Snowflake's cloud services and metadata layer make data sharing work. No data is moved or replicated during data sharing; the shared table refers to the original table and its micro-partitions. Changes in the source table are thus automatically visible in the shared table.</p>
</p>